# Tutorial: Extracting Wikipedia Pages & Detecting Redirects

Wikipedia provides large datasets of its contents via downloadable dumps. One common format is the .bz2-compressed pages-articles XML. This tutorial shows you how to read and parse this file using Python.

In [1]:
import bz2

# Define the path to the dump file - Update this with your path to the dump
dump_path = "D:Users/Paschalis/phd/data/dumps/UK/ukwiki-20250701-pages-articles-multistream.xml.bz2"

# Initialize a list to store the results
titles_and_texts = []

# Open and read the compressed XML file
with bz2.open(dump_path, 'rt', encoding='utf-8') as file:
    inside_page = False
    inside_text_tag = False
    title = ""
    text = []

    for line in file:
        line = line.strip()

        if line == "<page>":
            inside_page = True
            title = ""
            text = []
            inside_text_tag = False

        elif line == "</page>":
            if title and text:
                titles_and_texts.append({
                    "title": title,
                    "text": '\n'.join(text).strip()
                })
            inside_page = False

        elif inside_page and line.startswith("<title>") and line.endswith("</title>"):
            title = line[len("<title>"):-len("</title>")]

        elif inside_page and "<text" in line:
            inside_text_tag = True
            # Capture content after opening <text> tag (may include attributes)
            start = line.find('>') + 1
            end = line.find("</text>")
            if end != -1:
                text.append(line[start:end])
                inside_text_tag = False
            else:
                text.append(line[start:])

        elif inside_text_tag:
            end = line.find("</text>")
            if end != -1:
                text.append(line[:end])
                inside_text_tag = False
            else:
                text.append(line)

## Detecting Redirects

Redirects in the Wikipedia dump tend to start with a `#`.
For instance, a redirect in the dump can look like:
```
Redirect_name  -->  #REDIRECT Actual page title

In [2]:
if title and text:
    page_text = '\n'.join(text).strip()
    is_redirect = page_text.lower().startswith("#")
    titles_and_texts.append({
        "title": title,
        "text": page_text,
        "is_redirect": is_redirect
    })
    
# print a few of them to check them
for entry in titles_and_texts[:100]: # adjust the number here if you want to print less redirects
    print(f"Title: {entry['title']}")
    print(f"Text snippet: {entry['text'][:100]}...\n")

Title: Esperanto
Text snippet: #REDIRECT [[Есперанто]]...

Title: Wikipedia
Text snippet: #REDIRECT [[Вікіпедія]]...

Title: HomePage
Text snippet: #перенаправлення [[Головна сторінка]]...

Title: Географія
Text snippet: {{Географія вертикальний |image=Physical world.jpg |caption=Фізична карта світу}}
'''Геогра́фія,''' ...

Title: Атом
Text snippet: {{Оновити-ДС}}{{Otheruses|Атом (значення)}}
[[Файл:Діаграма Атома.svg|300px|обрамити|праворуч|Планет...

Title: Мільярд
Text snippet: '''Мільярд''' ({{lang-en|billion}}, {{lang-fr|milliard}})&amp;nbsp;— [[число]], назва величини &lt;m...

Title: Ядро
Text snippet: {{disambig}}
'''Ядро́'''&amp;nbsp;— основна частина, група певного колективу, яка визначає, організу...

Title: Мільйон
Text snippet: [[Файл:Rzz 1000 mille x 1000.svg|thumb|ліворуч|100пкс|Римський мільйон]]
[[Image:Visualisation 1 mil...

Title: Хімія
Text snippet: '''Хі́мія''' або '''хе́мія'''&lt;ref&gt;Глосарій термінів з хімії // Й.Опейда, О.Швайка. Ін-т фізико...

Title: Діаме

## Detect pages that you might want to remove

Our goal was to remove pages that act as automatic templates or are mostly help pages or list of links. To detect pages that adhere to this type you need to perform a manual check of titles.

What we noticed when we were building WikiTextGraph was that pages that have a colon (":") usually fit to the description above. These titles can vary from language to language, thus it can take some time to spot them.

Below we provide an example to see how could you look for pages like these.

In [ ]:
redirects = list()
for entry in titles_and_texts: # use list indexing if you don't want to overload your memory (e.g., [:1000], [1000:100000])
    text = entry["text"].strip()
    # if "значення" in entry["title"]: # this is to look for a pattern within the title
    if note text.startswith("#"): # this is to find redicts
        continue # skip non-redirects
    # if entry["title"].startswith("Користувач:"): # this is an alternative way to look if a title of a page starts with a specific pattern
        # refer to LANG_SETTINGS.yml for some examples in multiple language versions
        # print("Title: ", entry["title"])
        # print("Text: ", entry["text"][:20])

In [44]:
weird_titles = list()
for entry in titles_and_texts: # use list indexing if you don't want to overload your memory (e.g., [:1000], [1000:100000])
    # if "значення" in entry["title"]: # this is to look for a pattern within the title
    # if entry["text"].startswith("# "): # this is to find redicts
    # if entry["title"].startswith("Користувач:"): # this is an alternative way to look if a title of a page starts with a specific pattern
    if "значення" in entry["title"]:
        # refer to LANG_SETTINGS.yml for some examples in multiple language versions
        # print("Title: ", entry["title"])
        # print("Text: ", entry["text"][:20])
        # weird_title = entry["title"].split(":", maxsplit=1)[0]
        # weird_titles.append(weird_title)
        print(entry["title"])

Електрон (значення)
Кіно (значення)
Сон (значення)
Теннессі (значення)
Нью-Йорк (значення)
ФРН (значення)
Костянтинівка (значення)
Чернігівський район (значення)
Золочівський район (значення)
Красний Луч (значення)
Миколаївський район (значення)
Крим (значення)
Повість минулих літ (значення)
Аквілон (значення)
Альціона (значення)
Атланта (значення)
Аякс (значення)
Венера (значення)
Кипарис (значення)
Клеопатра (значення)
Мета (значення)
Смірна (значення)
Мусей (значення)
Помпа (значення)
Скит (значення)
Талас (значення)
Тантал (значення)
Термін (значення)
Хлорида (значення)
Олександрія (значення)
Олімпія (значення)
Ворошиловград (значення)
ПАР (значення)
Брижі (значення)
Крик (значення)
Тайфун (значення)
Ураган (значення)
Циклон (значення)
Заперечення (значення)
Паскаль (значення)
Порт (значення)
Реквізит (значення)
Шлях (значення)
Депресія (значення)
Колонія (значення)
Партія (значення)
Стратегія (значення)
Портал (значення)
ІТ (значення)
Ізоморфізм (значення)
Сервер (значення)
Землі 

In [ ]:
# for title in weird_titles[:500]:
#     print(title)
weird_titles_set = set(weird_titles)